In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# Assuming your CSV data is saved as 'data.csv'
data = pd.read_csv('data_processing/output.csv')

/Users/williamho/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Preprocessing
# One-hot encoding for categorical variables
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(data[['curr_attack', 'interrupted']])
encoded_labels = pd.get_dummies(data['mtd'])

# Combine one-hot encoded features with other numerical features
features = pd.concat([pd.DataFrame(encoded_features), data[['mtd_freq', 'compromised_num']]], axis=1)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

In [3]:
X_train, y_train

(       0    1    2    3    4    5    6    7  mtd_freq  compromised_num
 79   0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005030              120
 217  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005001              383
 113  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005015              173
 140  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.005004              243
 122  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.005004              192
 ..   ...  ...  ...  ...  ...  ...  ...  ...       ...              ...
 106  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005019              161
 14   0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.005129                5
 92   0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.005024              144
 179  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005001              316
 102  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.005008              158
 
 [176 rows x 10 columns],
      CompleteTopologyShuffle  IPShuffle  OSDiversity  ServiceDiversity
 79                     False       

In [4]:
# Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Output layer: 4 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


/Users/williamho/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.2348 - loss: 8.5044 - val_accuracy: 0.2500 - val_loss: 3.3500
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2874 - loss: 2.6305 - val_accuracy: 0.3611 - val_loss: 2.3811
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2460 - loss: 2.6118 - val_accuracy: 0.1944 - val_loss: 3.2275
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2933 - loss: 2.1027 - val_accuracy: 0.2500 - val_loss: 1.9372
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4098 - loss: 1.9764 - val_accuracy: 0.2500 - val_loss: 4.7158
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3422 - loss: 3.1225 - val_accuracy: 0.6389 - val_loss: 2.3125
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3077 - loss: 2.6512 - val_accuracy: 0.4722 - val_loss: 1.6956
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3639 - loss: 1.9578 - val_accuracy: 0.

In [6]:
# Evaluate the model
model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5871 - loss: 0.7589


[0.7823486328125, 0.5681818127632141]

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


data = pd.read_csv('data_processing/testing_dataset.csv')


X = data.drop('name', axis = 1)
y = data['name']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Output layer with 4 units for 4 classes and softmax activation

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.2)




Epoch 1/10


/Users/williamho/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.3959 - loss: 1.9433 - val_accuracy: 0.3958 - val_loss: 1.2223
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5520 - loss: 1.1010 - val_accuracy: 0.6917 - val_loss: 1.0697
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6686 - loss: 0.9968 - val_accuracy: 0.6917 - val_loss: 1.0372
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7231 - loss: 0.9140 - val_accuracy: 0.6917 - val_loss: 0.8613
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7456 - loss: 0.7824 - val_accuracy: 0.6917 - val_loss: 0.8177
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7361 - loss: 0.6988 - val_accuracy: 0.6917 - val_loss: 0.7679
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6994 - loss: 0.7026 - val_accuracy: 0.6917 - val_loss: 0.8043
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7326 - loss: 0.6327 - val_accuracy: 0.6917 - val_loss: 0.

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7416 - loss: 0.5634 
Test loss: 0.5620445013046265, Test accuracy: 0.75
